In [2]:
# encoding: utf-8
import time
import sys,io
import re
time1 = time.time()
import urllib, base64
import json


def get_token(API_Key,Secret_Key):
    host = 'https://aip.baidubce.com/oauth/2.0/token?grant_type=client_credentials&client_id='+API_Key+'&client_secret='+Secret_Key
    request = urllib.request.Request(host)
    request.add_header('Content-Type', 'application/json; charset=UTF-8')
    response = urllib.request.urlopen(request)
    content = response.read()
    content_json=json.loads(content)
    access_token=content_json['access_token']
    return access_token

def recognition_word_high(filepath,filename,access_token):
    url='https://aip.baidubce.com/rest/2.0/ocr/v1/accurate?access_token=' + access_token
    f = open(filepath + filename, 'rb')
    img = base64.b64encode(f.read())
    params = {"image": img}
    params = urllib.parse.urlencode(params).encode(encoding='UTF8')
    request = urllib.request.Request(url, params)
    request.add_header('Content-Type', 'application/x-www-form-urlencoded')
    response = urllib.request.urlopen(request)
    content = response.read()
    if (content):
        
        #log_id=re.findall('"log_id":"(.*?)"}',str(content),re.S)
        location=re.findall('"location": {(.*?)}',str(content),re.S)
        word=re.findall('"words": "(.*?)"}',str(content),re.S)
        for each_word in word:
            print (each_word.encode().decode())
        for each_location in location:
            print (each_location.encode().decode())
    #url_1='https://aip.baidubce.com/rest/2.0/solution/v1/form_ocr/get_request_result?access_token=' + access_token
    #params_1 = {"request_id": request_id[0],"request_type":"excel"}
    #params_1 = urllib.parse.urlencode(params_1).encode(encoding='UTF8')
    #request_1 = urllib.request.Request(url_1, params_1)
    #request_1.add_header('Content-Type', 'application/x-www-form-urlencoded')
    #response_1 = urllib.request.urlopen(request_1)
    #content_1 = response_1.read()
    #if (content_1):
        #print(content_1.decode())
    return word,location


if __name__ == '__main__':
    API_Key = "v6cd6adVIU9WlfSFB2XNiu95"
    Secret_Key = "CQ033fD5YdfArKI96sEBQOqGFBlDjSZC"
    filepath = "./tests/data/"
    filename="contract3 1.png"
    access_token=get_token(API_Key,Secret_Key)
    word,location=recognition_word_high=recognition_word_high(filepath,filename,access_token)

 Australian
 National
 niversity
 INDEPENDENT STUDY CONTRACT
 Note: Enrolment is subject to approval by the projects co-ordinator
 SECTION A(Students and Supervisors)
 UnilD:
 SURNAME:
 FIRST NAMES:
 PROJECT SUPERVISOR (may be external):
 COURSE SUPERVISOR ( RSCS academic):
 COURSE CODE, TITLE AND UNIT:
 SEMESTERS\xe2\x96\xa1S2Ye:
 PROJECT TITLE:
 LEARNING OBJECTIVES:
 PROJECT DESCRIPTION:
 Research School of Computer Science
 Form updated Jan 2017
"width": 93, "top": 156, "left": 330, "height": 22
"width": 67, "top": 173, "left": 339, "height": 23
"width": 66, "top": 196, "left": 355, "height": 19
"width": 626, "top": 262, "left": 315, "height": 40
"width": 546, "top": 318, "left": 345, "height": 27
"width": 385, "top": 379, "left": 182, "height": 28
"width": 57, "top": 429, "left": 174, "height": 29
"width": 102, "top": 480, "left": 173, "height": 25
"width": 114, "top": 488, "left": 555, "height": 21
"width": 304, "top": 529, "left": 173, "height": 30
"width": 294, "top": 567, "left

In [3]:
import numpy as np
top_list=[]
left_list=[]
width_list=[]
height_list=[]
all_info=re.findall(r'\d+\d*',str(location))
for i in range(len(location)):
    width_list.append(all_info[4*i])
    top_list.append(all_info[4*i+1])
    left_list.append(all_info[4*i+2])
    height_list.append(all_info[4*i+3])
top_list=np.array(top_list).astype(int)
left_list=np.array(left_list).astype(int)
width_list=np.array(width_list).astype(int)
height_list=np.array(height_list).astype(int)
print(top_list)
print(left_list)
print(width_list)
print(height_list)

[ 156  173  196  262  318  379  429  480  488  529  567  621  676  711
  860  992 1436 1443]
[330 339 355 315 345 182 174 173 555 173 173 172 182 183 183 183 163 868]
[ 93  67  66 626 546 385  57 102 114 304 294 260 282 125 191 188 286 175]
[22 23 19 40 27 28 29 25 21 30 33 25 28 24 18 22 26 19]


In [6]:
import cv2
img=cv2.imread("./tests/data/contract3 1.png")
cv2.imshow('visualize',img)
cv2.waitKey(0)
GrayImage=cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
ret, th = cv2.threshold(GrayImage,0,255,cv2.THRESH_BINARY | cv2.THRESH_OTSU)
#th_morph = cv2.morphologyEx(th, cv2.MORPH_CLOSE, cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (1, 1)))
th_blur=cv2.GaussianBlur(th,(5,5),0)
#th = cv2.adaptiveThreshold(GrayImage,255,cv2.ADAPTIVE_THRESH_MEAN_C,cv2.THRESH_BINARY,5,5)
#cv2.imwrite("./tests/data/contract3 1 th.png", th)
canny_result=cv2.Canny(th_blur,50,200)
cv2.imshow("canny_result",canny_result)
cv2.waitKey(0)
contours,hierarchy=cv2.findContours(canny_result,cv2.RETR_TREE,cv2.CHAIN_APPROX_NONE)
for cont in contours:
    perimeter = cv2.arcLength(cont,True)
    epsilon = 0.02*cv2.arcLength(cont,True)
    approx = cv2.approxPolyDP(cont,epsilon,True)
    if (len(approx)==4 and abs(cv2.contourArea(cont))>100):
        point_1=approx[0,0]
        point_2=approx[1,0]
        point_3=approx[2,0]
        point_4=approx[3,0]
        min_1_2_3=min(abs(Angle_Cosine(point_1,point_2,point_3)),abs(Angle_Cosine(point_2,point_1,point_3)),abs(Angle_Cosine(point_3,point_1,point_2)))
        min_1_2_4=min(abs(Angle_Cosine(point_1,point_2,point_4)),abs(Angle_Cosine(point_2,point_1,point_4)),abs(Angle_Cosine(point_4,point_1,point_2)))
        min_1_3_4=min(abs(Angle_Cosine(point_1,point_3,point_4)),abs(Angle_Cosine(point_3,point_1,point_4)),abs(Angle_Cosine(point_4,point_1,point_3)))
        min_2_3_4=min(abs(Angle_Cosine(point_2,point_3,point_4)),abs(Angle_Cosine(point_3,point_2,point_4)),abs(Angle_Cosine(point_4,point_2,point_3)))
        if(min_1_2_3<0.1 and min_1_2_4<0.1 and min_1_3_4<0.1 and min_2_3_4<0.1):
            cv2.drawContours(img,cont,-1,(255,0,0))
for i in range(len(top_list)):
    cv2.rectangle(img, (left_list[i],top_list[i]), (left_list[i]+width_list[i],top_list[i]+height_list[i]),(0,0,255), 2)
cv2.imshow("visualize", img)
cv2.waitKey(0)
cv2.imwrite("Visualize_Result.png", img)

True

In [1]:
def Angle_Cosine(a,b,c):
    a_b_vector=a-b
    a_c_vector=a-c
    cosine=(a_b_vector@a_c_vector)/(np.linalg.norm(a_b_vector)*np.linalg.norm(a_c_vector))
    return cosine